In [ ]:
from pathlib import Path
import os
import torch as pt
from torch import flatten
from flowtorch.analysis import SVD
import random
import matplotlib.pyplot as plt
from os.path import join

random.seed(10)
plt.rcParams["figure.dpi"] = 180

DATA_PATH = join(Path(os.path.abspath('')).parent, "data", "SVD")
OUTPUT_PATH = join(Path(os.path.abspath('')).parent, "output", "SVD")

- reduce